In [1]:
from os import environ
from getpass import getpass
from mysql.connector import connect, Error

admin_user = environ.get('MySQL_User')
admin_password = environ.get('MySQL_psswrd')


def test_connection(user=admin_user,password=admin_password):
    try:
        return connect(host="localhost",user=user,password=password,database="online_movie_rating")
    except Error as e:
        print("It was not possible to connect with the database:  ",e)
        return None          
            
def search_user(user):
    global cursor
    cursor.execute("""SELECT id FROM reviewers WHERE user_name = '%s'""" %user)
    user_id = cursor.fetchone()
    if user_id != None:
        return user_id[0]
    else:
        return used_id

def search_movie(title,year=None):
    global cursor
    if year == None:
        cursor.execute("""SELECT id,release_year FROM movies WHERE title = '%s'""" %title)
        movies = cursor.fetchall()
        return movies
    else:
        cursor.execute("""SELECT id FROM movies WHERE (title,release_year) = ('%s',%s)""" %(title,year))
        movie_id = cursor.fetchone()
        if movie_id != None:
            return movie_id[0]
        else:
            return movie_id
    
def search_genre(genre):
    global cursor
    cursor.execute("""SELECT id FROM genres WHERE genre = '%s'""" %genre)
    genre_id = cursor.fetchone()
    if genre_id != None:
        return genre_id[0]
    else:
        return genre_id
    
def search_rating(movie_id,reviewer_id):
    global cursor
    cursor.execute("""SELECT rating FROM ratings WHERE (movie_id,reviewer_id) = (%s,%s)""" %(movie_id,reviewer_id))
    rating = cursor.fetchone()
    if rating != None:
        return rating[0]
    else:
        return rating

def insert_reviewer(user_name,user_firstname,user_lastname):
    global cursor
    global connection
    cursor.execute("""INSERT INTO reviewers (user_name,first_name, last_name) 
            VALUES ( "%s", "%s", "%s")""" %(user_name,user_firstname,user_lastname))
    connection.commit()
    print ("Welcome %s!!! Your user account has been created." %user_name)
    return

def insert_genre(genre):
    global cursor
    global connection
    cursor.execute("""INSERT INTO genres (genre) VALUES ("%s")""" %genre)
    connection.commit()
    print ("The genre %s had to be added to the database since it did not previously exist." %genre)
    return

def insert_movie(title,year,genre_id=None):
    global cursor
    global connection
    if genre_id == None:
        cursor.execute("""INSERT INTO movies (title,release_year) VALUES ("%s",%s)""" %(title,year))
    else:
        cursor.execute("""INSERT INTO movies (title,release_year,genre_id) VALUES ("%s",%s,%s)""" %(title,year,genre_id))
    connection.commit()
    print ("The movie %s (%s) has been succesfully stored in the database." %(title,year))
    return
    
def insert_rating(movie_id,reviewer_id,rating):
    global cursor
    global connection
    cursor.execute("""INSERT INTO ratings (movie_id,reviewer_id,rating) VALUES (%s,%s,%s)""" %(movie_id,reviewer_id,rating))
    connection.commit()
    print ("Your rating for %s has been succesfully stored in the database." %movie_title)
    return

def edit_rating(movie_id,reviewer_id,rating):
    global cursor
    global connection
    cursor.execute("""UPDATE ratings SET rating = %s WHERE (movie_id,reviewer_id) = (%s,%s)""" %(rating,movie_id,reviewer_id))
    connection.commit()
    print ("Your rating for %s has been edited to %s." %(movie_title,new_rating))
    return




user = input ("Enter your username: ")


with test_connection() as connection:
    with connection.cursor() as cursor:
        
        #### USER CHECKS ####
        
        user_id=search_user(user)
        if user_id != None:
            print ("Welcome back %s." %user)
        else:
            print ("Hi %s, it seems like you are not registered as reviewer.")
            print (" Let's fill in your personal details and create a user account for you")
            user_firstname = input ("What's your firstname?:")
            user_lastname = input ("What's your lastname?:")
            insert_reviewer(user,user_firstname,user_lastname)
            user_id=search_user(user)
            
            
        
        #### MOVIE CHECKS ####
        
        response = None ; iter = 0
        while response != 'yes' and response != 'y' and response != 'no' and response != 'n' and iter < 3:
            
            response = input ("Do you want to rate a movie?: (yes/y or no/n) ")
            
            if response == 'yes' or response == 'y':
                movie_id = None ; title_iter = 0
                check_year = None ; year_iter = 0
                while movie_id == None and title_iter < 3:
                    movie_title = input ("Which movie do you want to rate?: ")
                    movies = search_movie(movie_title)
                    
                    if len(movies) > 1:
                        print("MANY")
                        while check_year != True and year_iter < 3:
                            movie_year=input("There is more than one movie with that title. Could you especify the year of release of the movie you want to rate?: ")
                            for movie in movies:
                                if movie[1] == int(movie_year):
                                    movie_id = movie[0]
                                    check_year = True
                            if movie_id == None:
                                
                                response_year = None ; resp_year_iter = 0
                                while response_year == None and resp_year_iter < 3: 
                                    response_year = input ("There is no movie in the database with that title and year of release. Are you sure %s is the year of release of the movie?: (yes/y or no/n) " %movie_year)   
                                    if response_year == "yes" or response_year == "y":
                                        check_year = True
                                    elif response_year == "no" or response_year == "n":
                                        year_iter += 1
                                    else:
                                        print ("That's not a valid answer.")
                                        response_year = None
                                        resp_year_iter += 1
                                        if resp_year_iter >= 3:
                                            print ("Sorry, but I can't help you if you don't answer my question as requested.")   
                        
                        if movie_id == None and check_year == True:
                            
                            response_title = None ; resp_title_iter = 0
                            while response_title == None and resp_title_iter < 3:
                                response_title = input ("There is no movie in the database with that title and year of release. Are you sure '%s' is the title of the movie?: (yes/y or no/n) " %movie_title)
                                if response_title == "yes" or response_title == "y":
                                    movie_id = True
                                elif response_title == "no" or response_title == "n":
                                    title_iter += 1
                                else:
                                    print ("That's not a valid answer.")
                                    response_title = None
                                    resp_title_iter += 1
                                    if resp_title_iter >= 3:
                                        print ("Sorry, but I can't help you if you don't answer my question as requested.")
                            
                    elif len(movies) == 0:
                        print("NONE")
                        response_title = None ; resp_title_iter = 0 
                        while response_title == None and resp_title_iter < 3:
                            response_title = input ("There is no movie in the database with that title. Are you sure '%s' is the title of the movie?: (yes/y or no/n) " %movie_title)
                            if response_title == "yes" or response_title == "y":
                                movie_id = True
                                
                                check_year = None ; year_iter = 0
                                while check_year != True and year_iter < 3:
                                    movie_year = input ("What's the year of release of the movie?: ")
                                    
                                    response_year = None ; resp_year_iter = 0
                                    while response_year == None and resp_year_iter < 3: 
                                        response_year = input ("Are you sure %s is the year of release of the movie?: (yes/y or no/n) " %movie_year)   
                                        if response_year == "yes" or response_year == "y":
                                            check_year = True
                                        elif response_year == "no" or response_year == "n":
                                            year_iter += 1
                                            movie_id = False
                                        else:
                                            print ("That's not a valid answer.")
                                            response_year = None
                                            resp_year_iter += 1
                                            if resp_year_iter >= 3:
                                                print ("Sorry, but I can't help you if you don't answer my question as requested.")

                                
                            elif response_title == "no" or response_title == "n":
                                title_iter += 1
                            else:
                                print ("That's not a valid answer.")
                                response_title = None
                                resp_title_iter += 1
                                if resp_title_iter >= 3:
                                    print ("Sorry, but I can't help you if you don't answer my question as requested.")
                                    
                    else:
                        print("ONE")
                        check_year = None ; year_iter = 0
                        while check_year != True and year_iter < 3:
                            movie_year = input ("What's the year of release of the movie?: ")
                            if movies[0][1] == int (movie_year):
                                movie_id = movies[0][0]
                                check_year = True
                            else:
                                
                                response_year = None ; resp_year_iter = 0
                                while response_year == None and resp_year_iter < 3: 
                                    response_year = input ("There is no movie in the database with that title and year of release. Are you sure %s is the year of release of the movie?: (yes/y or no/n) " %movie_year)   
                                    if response_year == "yes" or response_year == "y":
                                        check_year = True
                                    elif response_year == "no" or response_year == "n":
                                        year_iter += 1
                                        movie_id = False
                                    else:
                                        print ("That's not a valid answer.")
                                        response_year = None
                                        resp_year_iter += 1
                                        if resp_year_iter >= 3:
                                            print ("Sorry, but I can't help you if you don't answer my question as requested.")
                                
                        if movie_id == None and check_year == True:
                            
                            response_title = None ; resp_title_iter = 0
                            while response_title == None and resp_title_iter < 3:
                                response_title = input ("There is no movie in the database with that title and year of release. Are you sure '%s' is the title of the movie?: (yes/y or no/n) " %movie_title)
                                if response_title == "yes" or response_title == "y":
                                    movie_id = True
                                elif response_title == "no" or response_title == "n":
                                    title_iter += 1
                                else:
                                    print ("That's not a valid answer.")
                                    response_title = None
                                    resp_title_iter += 1
                                    if resp_title_iter >= 3:
                                        print ("Sorry, but I can't help you if you don't answer my question as requested.")

            #### NEW ENTRY ####
                if movie_id != None:
                    if check_year != None:                                 
                        if movie_id is True:
                            print ("Since you have confirmed the title and year of release of the movie you want to rate, let's register the movie in our database")
                            genre = input ("What's the genre of the movie?: (Leave empty if unknown) ")
                            if genre == "":
                                insert_movie(movie_title,movie_year)
                                movie_id = search_movie(movie_title,movie_year)
                            else:
                                genre_id = search_genre(genre)
                                if genre_id == None:                                
                                    insert_genre(genre)
                                    genre_id = search_genre(genre)
                                insert_movie(movie_title,movie_year,genre_id)
                                movie_id = search_movie(movie_title,movie_year)
                        print(movie_id,user_id)
                        prev_rating = search_rating(movie_id,user_id)
                        print(prev_rating)
                        if prev_rating != None:
                            response_rating = None ; resp_rating_iter = 0
                            while response_rating == None and resp_rating_iter < 3:
                                response_rating = input ("You have already rate this movie with a %s. Do you want to edit your rating?: (yes/y or no/n) " %prev_rating)
                                if response_rating == "yes" or response_rating == "y":
                                    new_rating = input ("What rating do you want to give to the movie?: ")
                                    edit_rating(movie_id,user_id,new_rating)
                                    print ("See you next time")
                                elif response_rating == "no" or response_rating == "n":
                                    print ("See you next time then. Bye.")
                                else:
                                    print ("That's not a valid answer.")
                                    response_rating = None
                                    resp_rating_iter += 1
                                    if resp_rating_iter >= 3:
                                        print ("Sorry, but I can't help you if you don't answer my question as requested.")
                        else:
                            rating = input ("What rating do you want to give to the movie?: ")
                            insert_rating(movie_id,user_id,rating)
                            print ("See you next time")
                    else: 
                        print ("Since you are not sure about the year of release of the movie I can't help you. See you next time.")
                else:
                    print ("Since you are not sure about the title of the movie I can't help you. See you next time.")
                
                    
            elif response == 'no' or response == 'n':
                print ("See you next time then. Bye.")
            
            else:
                print ("That's not a valid answer.")
                iter += 1
                if iter >= 3:
                    print ("Sorry, but I can't help you if you don't answer my question as requested.")
                    print ("See you next time.")

Enter your username: pablos-p
Welcome back pablos-p.
Do you want to rate a movie?: (yes/y or no/n) y
Which movie do you want to rate?: Braveheart
ONE
What's the year of release of the movie?: 1925
There is no movie in the database with that title and year of release. Are you sure 1925 is the year of release of the movie?: (yes/y or no/n) y
There is no movie in the database with that title and year of release. Are you sure 'Braveheart' is the title of the movie?: (yes/y or no/n) y
Since you have confirmed the title and year of release of the movie you want to rate, let's register the movie in our database
What's the genre of the movie?: (Leave empty if unknown) 
The movie Braveheart (1925) has been succesfully stored in the database.
36 23
None
What rating do you want to give to the movie?: 0.0
Your rating for Braveheart has been succesfully stored in the database.
See you next time
